In [1]:
import os
from regex import F
import yaml
import torch
from torch.utils.data import Dataset
from torchvision.io import read_video

class V2VBenchDataset(Dataset):
    def __init__(self, config_path, videos_dir):
        """
        Args:
            config_path (str): Path to config.yaml file
            videos_dir (str): Directory containing video files
        """
        self.videos_dir = videos_dir
        
        # Load config file
        with open(config_path, 'r') as f:
            self.config = yaml.safe_load(f)
        
        # Flatten the data structure
        self.samples = []
        for video_data in self.config['data']:
            video_id = video_data['video_id']
            source_prompt = video_data['prompt']
            video_path = os.path.join(self.videos_dir, f"{video_id}.mp4")
            
            for edit in video_data['edit']:
                target_prompt = edit['prompt']
                self.samples.append({
                    'video_path': video_path,
                    'video_id': video_id,
                    'source_prompt': source_prompt,
                    'target_prompt': target_prompt,
                    'src_words': edit.get('src_words', ''),
                    'tgt_words': edit.get('tgt_words', ''),
                    'edit_type': edit.get('type', '')
                })
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # Load video (frames, audio, info)
        # Note: You may need to adjust this based on your video format and needs
        video, _, info = read_video(sample['video_path'], pts_unit='sec')
        
        # Convert video to float and normalize to [0, 1]
        video = video.float() / 255.0
        
        return {
            'video': video,
            'video_id': sample['video_id'],
            'source_prompt': sample['source_prompt'],
            'target_prompt': sample['target_prompt'],
            'src_words': sample['src_words'],
            'tgt_words': sample['tgt_words'],
            'edit_type': sample['edit_type']
        }

# Example usage
if __name__ == "__main__":
    config_path = "/data/chx/V2VBench/config.yaml"
    videos_dir = "/data/chx/V2VBench/videos"
    
    dataset = V2VBenchDataset(config_path, videos_dir)
    
    # Create a DataLoader
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        num_workers=4
    )
    
    # Iterate through the dataset
    for batch in dataloader:
        print(f"Batch size: {len(batch['video'])}")
        print(f"Video ID: {batch['video_id']}")
        print(f"Video shape: {batch['video'].shape}")
        print(f"Source prompt: {batch['source_prompt']}")
        print(f"Target prompt: {batch['target_prompt']}")
        print("---")

Batch size: 1
Video ID: ['aurora']
Video shape: torch.Size([1, 24, 512, 512, 3])
Source prompt: ['green and blue aurora paints the night sky over mountain silhouettes']
Target prompt: ['red and yellow aurora paints the night sky over mountain silhouettes']
---
Batch size: 1
Video ID: ['aurora']
Video shape: torch.Size([1, 24, 512, 512, 3])
Source prompt: ['green and blue aurora paints the night sky over mountain silhouettes']
Target prompt: ['white and pure cloud paints the night sky over mountain silhouettes']
---
Batch size: 1
Video ID: ['aurora']
Video shape: torch.Size([1, 24, 512, 512, 3])
Source prompt: ['green and blue aurora paints the night sky over mountain silhouettes']
Target prompt: ['green and blue aurora paints the night sky over mountain silhouettes, van gogh starry night style']
---
Batch size: 1
Video ID: ['beam']
Video shape: torch.Size([1, 24, 512, 512, 3])
Source prompt: ['symmetric neon light tunnel with blue and purple hues']
Target prompt: ['symmetric neon light

In [ ]:
%run edit_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 25 \
    --infer-steps 30 \
    --inject 5 \
    --seed 42 \
    --embedded-cfg-scale 2 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results/V2VBench/rf-solver_2order \
    --dataset "V2VBench"

/data/chx/anaconda3/envs/HunyuanVideo-std/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-04 17:00:36.666 | INFO     | hyvideo.inference:from_pretrained:155 - Got text-to-video model root path: ckpts
2025-04-04 17:00:36.667 | INFO     | hyvideo.inference:from_pretrained:190 - Building model...


detect you are not use the latest yunchang. Please install yunchang>=0.4.0
{'apply_final_norm': False,
 'batch_size': 1,
 'cfg_scale': 1.0,
 'dataset': 'V2VBench',
 'denoise_type': 'flow',
 'disable_autocast': False,
 'dit_weight': '/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt',
 'embedded_cfg_scale': 2.0,
 'feature_path': 'feature',
 'flow_reverse': True,
 'flow_shift': 7.0,
 'flow_solver': 'euler',
 'hidden_state_skip_layer': 2,
 'infer_steps': 2,
 'inject': 5,
 'inverse_video_path': None,
 'latent_channels': 16,
 'linear_schedule_end': 25,
 'load_key': 'module',
 'model': 'HYVideo-T/2-cfgdistill',
 'model_base': 'ckpts',
 'model_resolution': '540p',
 'name_suffix': '',
 'neg_prompt': None,
 'num_videos': 1,
 'precision': 'bf16',
 'prompt': None,
 'prompt_template': 'dit-llm-encode',
 'prompt_template_video': 'dit-llm-encode-video',
 'reproduce': False,
 'ring_degree': 1,
 'rope_theta': 256,
 'save_path': './results/V2VBench/rf

/home/chx/mySrc/HunyuanVideo/hyvideo/modules/fp8_optimization.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fp8_map = torch.load(fp8_map_path, map_location=lambda sto

loading vae


/home/chx/mySrc/HunyuanVideo/hyvideo/vae/__init__.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(vae_ckpt, map_location=vae.device)
2025-04-04 17:03:

...

100%|██████████| 2/2 [00:08<00:00,  4.05s/it]


cuda:0
cuda:0
sigmas: tensor([1.0000, 0.8750, 0.0000])
Using inversed latents
Step 0: t = 1000.0
Step 1: t = 875.0


Step 0: t_curr = 1000.0, t_prev = 875.0
t:2


Step 1: t_curr = 875.0, t_prev = 0.0
t:1


100%|██████████| 2/2 [00:06<00:00,  3.43s/it]
2025-04-04 19:01:37.202 | INFO     | hyvideo.inference:predict:709 - Success, time: 42.27493476867676
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-04-04 19:01:38.841 | INFO     | __main__:process_sample:157 - 保存编辑视频到: ./results/V2VBench/rf-solver_2order/woman-dog/edit_002.mp4
处理数据集批次: 100%|██████████| 150/150 [1:58:01<00:00, 47.21s/it]


In [ ]:
%run edit_video2.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 25 \
    --infer-steps 30 \
    --prompt "a man performing the moonwalk dance" \
    --target-prompt "a clown performing the moonwalk dance" \
    --inject 5 \
    --seed 42 \
    --embedded-cfg-scale 3.5 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results/rf-solver_2order \
    --inverse-video-path "/data/chx/V2VBench/videos/moonwalk.mp4"

In [3]:
%run edit_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 25 \
    --infer-steps 45 \
    --prompt "a man in a white shirt and black pants wearing gray sneakers performing the moonwalk dance" \
    --target-prompt "a iron man in iron man's Suit performing the moonwalk dance" \
    --inject 5 \
    --seed 42 \
    --embedded-cfg-scale 3.5 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results/rf-solver_2order \
    --inverse-video-path "/data/chx/V2VBench/videos/moonwalk.mp4"

2025-04-12 17:31:28.037 | INFO     | hyvideo.inference:from_pretrained:155 - Got text-to-video model root path: ckpts
2025-04-12 17:31:28.037 | INFO     | hyvideo.inference:from_pretrained:190 - Building model...


{'apply_final_norm': False,
 'batch_size': 1,
 'cfg_scale': 1.0,
 'dataset': None,
 'denoise_type': 'flow',
 'disable_autocast': False,
 'dit_weight': '/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt',
 'embedded_cfg_scale': 3.5,
 'feature_path': 'feature',
 'flow_reverse': True,
 'flow_shift': 7.0,
 'flow_solver': 'euler',
 'hidden_state_skip_layer': 2,
 'infer_steps': 45,
 'inject': 5,
 'inverse_video_path': '/data/chx/V2VBench/videos/moonwalk.mp4',
 'latent_channels': 16,
 'linear_schedule_end': 25,
 'load_key': 'module',
 'model': 'HYVideo-T/2-cfgdistill',
 'model_base': 'ckpts',
 'model_resolution': '540p',
 'name_suffix': '',
 'neg_prompt': None,
 'num_samples': None,
 'num_videos': 1,
 'precision': 'bf16',
 'prompt': 'a man in a white shirt and black pants wearing gray sneakers '
           'performing the moonwalk dance',
 'prompt_template': 'dit-llm-encode',
 'prompt_template_video': 'dit-llm-encode-video',
 'reproduce': Fa

2025-04-12 17:33:39.568 | INFO     | hyvideo.inference:load_state_dict:345 - Loading torch model /home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt...
2025-04-12 17:33:52.337 | INFO     | hyvideo.vae:load_vae:29 - Loading 3D VAE model (884-16c-hy) from: ./ckpts/hunyuan-video-t2v-720p/vae


loading vae


2025-04-12 17:33:55.647 | INFO     | hyvideo.vae:load_vae:55 - VAE to dtype: torch.float16
2025-04-12 17:33:55.652 | INFO     | hyvideo.text_encoder:load_text_encoder:28 - Loading text encoder model (llm) from: ./ckpts/text_encoder
Loading checkpoint shards: 100%|██████████| 4/4 [01:44<00:00, 26.10s/it]
2025-04-12 17:35:47.809 | INFO     | hyvideo.text_encoder:load_text_encoder:50 - Text encoder to dtype: torch.float16
2025-04-12 17:35:47.813 | INFO     | hyvideo.text_encoder:load_tokenizer:64 - Loading tokenizer (llm) from: ./ckpts/text_encoder
2025-04-12 17:35:48.679 | INFO     | hyvideo.text_encoder:load_text_encoder:28 - Loading text encoder model (clipL) from: ./ckpts/text_encoder_2
2025-04-12 17:35:57.709 | INFO     | hyvideo.text_encoder:load_text_encoder:50 - Text encoder to dtype: torch.float16
2025-04-12 17:35:57.712 | INFO     | hyvideo.text_encoder:load_tokenizer:64 - Loading tokenizer (clipL) from: ./ckpts/text_encoder_2
2025-04-12 17:35:58.057 | INFO     | __main__:main:7

...

 96%|█████████▌| 43/45 [03:28<00:09,  4.80s/it]

Step 43: t_curr = 245.61404418945312, t_prev = 137.25491333007812


 98%|█████████▊| 44/45 [03:33<00:04,  4.81s/it]

Step 44: t_curr = 137.25491333007812, t_prev = 0.0


100%|██████████| 45/45 [03:37<00:00,  4.84s/it]
2025-04-12 17:47:28.044 | INFO     | hyvideo.inference:predict:709 - Success, time: 689.5644915103912
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-04-12 17:47:30.620 | INFO     | __main__:process_sample:175 - 保存单次结果到: ./results/rf-solver_2order/moonwalk_single_video_20250412-174728_s42.mp4
2025-04-12 17:47:30.628 | INFO     | __main__:main:200 - 单视频处理完成


In [2]:
%run edit_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 53 \
    --infer-steps 30 \
    --prompt "A black swan swimming in a pond with lush greenery in the background." \
    --target-prompt "A majestic flamingo swimming in a pond with lush greenery in the background." \
    --inject 5 \
    --seed 42 \
    --embedded-cfg-scale 2 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results \
    --inverse-video-path "/data/chx/BalanceCC-rename/Result/Animal/blackswan.mp4"

2025-04-03 12:50:25.940 | INFO     | hyvideo.inference:from_pretrained:155 - Got text-to-video model root path: ckpts
2025-04-03 12:50:25.941 | INFO     | hyvideo.inference:from_pretrained:190 - Building model...


{'apply_final_norm': False,
 'batch_size': 1,
 'cfg_scale': 1.0,
 'denoise_type': 'flow',
 'disable_autocast': False,
 'dit_weight': '/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt',
 'embedded_cfg_scale': 2.0,
 'feature_path': 'feature',
 'flow_reverse': True,
 'flow_shift': 7.0,
 'flow_solver': 'euler',
 'hidden_state_skip_layer': 2,
 'infer_steps': 30,
 'inject': 5,
 'inverse_video_path': '/data/chx/BalanceCC-rename/Result/Animal/blackswan.mp4',
 'latent_channels': 16,
 'linear_schedule_end': 25,
 'load_key': 'module',
 'model': 'HYVideo-T/2-cfgdistill',
 'model_base': 'ckpts',
 'model_resolution': '540p',
 'name_suffix': '',
 'neg_prompt': None,
 'num_videos': 1,
 'precision': 'bf16',
 'prompt': 'A black swan swimming in a pond with lush greenery in the '
           'background.',
 'prompt_template': 'dit-llm-encode',
 'prompt_template_video': 'dit-llm-encode-video',
 'reproduce': False,
 'ring_degree': 1,
 'rope_theta': 256,
 

2025-04-03 12:52:20.378 | INFO     | hyvideo.inference:load_state_dict:345 - Loading torch model /home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt...
2025-04-03 12:53:03.710 | INFO     | hyvideo.vae:load_vae:29 - Loading 3D VAE model (884-16c-hy) from: ./ckpts/hunyuan-video-t2v-720p/vae


loading vae


2025-04-03 12:53:06.936 | INFO     | hyvideo.vae:load_vae:55 - VAE to dtype: torch.float16
2025-04-03 12:53:06.942 | INFO     | hyvideo.text_encoder:load_text_encoder:28 - Loading text encoder model (llm) from: ./ckpts/text_encoder
Loading checkpoint shards: 100%|██████████| 4/4 [00:42<00:00, 10.56s/it]
2025-04-03 12:53:54.599 | INFO     | hyvideo.text_encoder:load_text_encoder:50 - Text encoder to dtype: torch.float16
2025-04-03 12:53:54.602 | INFO     | hyvideo.text_encoder:load_tokenizer:64 - Loading tokenizer (llm) from: ./ckpts/text_encoder
2025-04-03 12:53:55.238 | INFO     | hyvideo.text_encoder:load_text_encoder:28 - Loading text encoder model (clipL) from: ./ckpts/text_encoder_2
2025-04-03 12:53:56.092 | INFO     | hyvideo.text_encoder:load_text_encoder:50 - Text encoder to dtype: torch.float16
2025-04-03 12:53:56.096 | INFO     | hyvideo.text_encoder:load_tokenizer:64 - Loading tokenizer (clipL) from: ./ckpts/text_encoder_2


...

 93%|█████████▎| 28/30 [03:13<00:13,  6.87s/it]

Step 28: t_curr = 333.3333435058594, t_prev = 194.44444274902344
t:2


 97%|█████████▋| 29/30 [03:20<00:06,  6.86s/it]

Step 29: t_curr = 194.44444274902344, t_prev = 0.0
t:1


100%|██████████| 30/30 [03:27<00:00,  6.92s/it]
2025-04-03 13:01:46.524 | INFO     | hyvideo.inference:predict:709 - Success, time: 469.08821153640747
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-04-03 13:01:49.657 | INFO     | __main__:main:89 - Sample save to: ./results/2025-04-03-13:01:46_seed42_A black swan swimming in a pond with lush greenery in the background..mp4


In [1]:
%run sample_video.py \
    --dit-weight "/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt" \
    --video-size 512 512 \
    --video-length 25 \
    --infer-steps 30 \
    --prompt "A cat walks on the grass, realistic style." \
    --seed 42 \
    --embedded-cfg-scale 6.0 \
    --flow-shift 7.0 \
    --flow-reverse \
    --use-cpu-offload \
    --use-fp8 \
    --save-path ./results

/data/chx/anaconda3/envs/HunyuanVideo-std/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-14 20:33:19.204 | INFO     | hyvideo.inference:from_pretrained:154 - Got text-to-video model root path: ckpts
2025-01-14 20:33:19.205 | INFO     | hyvideo.inference:from_pretrained:189 - Building model...


detect you are not use the latest yunchang. Please install yunchang>=0.4.0
Namespace(model='HYVideo-T/2-cfgdistill', latent_channels=16, precision='bf16', rope_theta=256, vae='884-16c-hy', vae_precision='fp16', vae_tiling=True, text_encoder='llm', text_encoder_precision='fp16', text_states_dim=4096, text_len=256, tokenizer='llm', prompt_template='dit-llm-encode', prompt_template_video='dit-llm-encode-video', hidden_state_skip_layer=2, apply_final_norm=False, text_encoder_2='clipL', text_encoder_precision_2='fp16', text_states_dim_2=768, tokenizer_2='clipL', text_len_2=77, denoise_type='flow', flow_shift=7.0, flow_reverse=True, flow_solver='euler', use_linear_quadratic_schedule=False, linear_schedule_end=25, model_base='ckpts', dit_weight='/home/chx/mySrc/HunyuanVideo/ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states_fp8.pt', model_resolution='540p', load_key='module', use_cpu_offload=True, batch_size=1, infer_steps=30, disable_autocast=False, save_path='./results', save

/home/chx/mySrc/HunyuanVideo/hyvideo/modules/fp8_optimization.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fp8_map = torch.load(fp8_map_path, map_location=lambda sto

loading vae


/home/chx/mySrc/HunyuanVideo/hyvideo/vae/__init__.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(vae_ckpt, map_location=vae.device)
2025-01-14 20:36:

Enable sequential CPU offload.


2025-01-14 20:36:15.189 | DEBUG    | hyvideo.inference:predict:650 - 
                        height: 512
                         width: 512
                  video_length: 25
                        prompt: ['A cat walks on the grass, realistic style.']
                    neg_prompt: ['Aerial view, aerial view, overexposed, low quality, deformation, a poor composition, bad hands, bad teeth, bad eyes, bad limbs, distortion']
                          seed: 42
                   infer_steps: 30
         num_videos_per_prompt: 1
                guidance_scale: 1.0
                      n_tokens: 7168
                    flow_shift: 7.0
       embedded_guidance_scale: 6.0


self._execution_device cuda:0
cuda:0
cuda:0
Step 0: t = 1000.0
Step 1: t = 995.0980834960938
Step 2: t = 989.8989868164062
Step 3: t = 984.375
Step 4: t = 978.49462890625
Step 5: t = 972.22216796875
Step 6: t = 965.5172119140625
Step 7: t = 958.3333740234375
Step 8: t = 950.6172485351562
Step 9: t = 942.3078002929688
Step 10: t = 933.333251953125
Step 11: t = 923.611083984375
Step 12: t = 913.04345703125
Step 13: t = 901.51513671875
Step 14: t = 888.888916015625
Step 15: t = 875.0
Step 16: t = 859.6491088867188
Step 17: t = 842.5925903320312
Step 18: t = 823.5294189453125
Step 19: t = 802.0833129882812
Step 20: t = 777.77783203125
Step 21: t = 750.0
Step 22: t = 717.94873046875
Step 23: t = 680.5556030273438
Step 24: t = 636.3637084960938
Step 25: t = 583.3333740234375
Step 26: t = 518.5184936523438
Step 27: t = 437.5000305175781
Step 28: t = 333.3333435058594
Step 29: t = 194.44444274902344


100%|██████████| 30/30 [00:49<00:00,  1.64s/it]
2025-01-14 20:37:23.031 | INFO     | hyvideo.inference:predict:684 - Success, time: 67.84050464630127
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-01-14 20:37:24.919 | INFO     | __main__:main:57 - Sample save to: ./results/2025-01-14-20:37:23_seed42_A cat walks on the grass, realistic style..mp4
